In [1]:
import pandas as pd
import numpy as np
import seaborn as sn
import matplotlib.pyplot as plt


In [52]:
data = pd.read_csv('data/train_data.csv')
data.info()
data.head()
data.drop(labels=["BBLE", "LOT","PERIOD", "VALTYPE",
                  "Borough", "New Georeferenced Column",
                  "OWNER", "AVLAND2", "AVTOT2",
                  "EXLAND2", "EXTOT2","BIN", "EXCD1", "EXCD2",
                  "EXMPTCL"],axis=1, inplace=True)
# wywalono BBLE bo jest to sztuczny ID, LOT bo jest randomowo
# przydzielany przez miasto, PERIOD i VALTYPE wszystkie mają
# taką samą wartość
# BORO - kod dzielnicy, BLOCK - numer bloku, LOT-numer jednostki w bloku
# EASEMENT - co mozna a czego nie można robić, null
# BLDGCL - klasa budynku - np. dwupiętrowy dla biznesu itp
# TAXCLASS - klasa podatkowa, uproszczony building class 1-mieszkalne  <=mieszkania
# 2a -4-6, 2b-7-10 (a i b mieszkania na wynajem) 2c-2-10 na współdzielnie
# 2 >=11, 3-wiekszosc utility property, 4-cala reszta komercyjna
# LTFRONT - jak szeroki od strony ulicy lot
# LTDEPTH - jak jest głeboki lot
# EXT - 
# STORIES - ilośc pięter
# EXCD1 - excemption code 1 - 
# EXMPTCL - excempt class
# EXCD2 - 
# AVLAND2, AVTOT2, EXLAND2, EXTOT2 wywalamy bo to zmiana w ciagu roku
# borough - nazwa dzielnicy w stringu
# community board  - 
# council district - 
# census tract - 
# BIN - nwm to usuwam bo nwm jak uzupełnić



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11946 entries, 0 to 11945
Data columns (total 40 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   BBLE                      11946 non-null  object 
 1   BORO                      11946 non-null  int64  
 2   BLOCK                     11946 non-null  int64  
 3   LOT                       11946 non-null  int64  
 4   EASEMENT                  126 non-null    object 
 5   OWNER                     11672 non-null  object 
 6   BLDGCL                    11946 non-null  object 
 7   TAXCLASS                  11946 non-null  object 
 8   LTFRONT                   11946 non-null  int64  
 9   LTDEPTH                   11946 non-null  int64  
 10  EXT                       2982 non-null   object 
 11  STORIES                   11235 non-null  float64
 12  FULLVAL                   11946 non-null  int64  
 13  AVLAND                    11946 non-null  int64  
 14  AVTOT 

In [53]:
round(data.isnull().sum()/len(data), 5)

BORO                0.00000
BLOCK               0.00000
EASEMENT            0.98945
BLDGCL              0.00000
TAXCLASS            0.00000
LTFRONT             0.00000
LTDEPTH             0.00000
EXT                 0.75038
STORIES             0.05952
FULLVAL             0.00000
AVLAND              0.00000
AVTOT               0.00000
EXLAND              0.00000
EXTOT               0.00000
STADDR              0.00142
POSTCODE            0.02947
BLDFRONT            0.00000
BLDDEPTH            0.00000
YEAR                0.00000
Latitude            0.03633
Longitude           0.03633
Community Board     0.03633
Council District    0.03633
Census Tract        0.03633
NTA                 0.03633
dtype: float64

In [46]:
# XGBOOST, feature importances
print(data["EASEMENT"].unique())

[nan 'E' 'F' 'G' 'H' 'N' 'I']


In [51]:
data[data.EXMPTCL.isnull()]
# ext na dwie kolumny e i g
# GEOGRAFICZNIE UZUPELNIAMY NA PODSTAWIE BLOKÓW
# STORIES - BIERZEMY SREDNIA Z BLOKOW
# EXCEMPTION CODES NULLE NA 0
# EASEMENT - DWIE KOLUMNY E I N
# obliczyc powierzchnie i normalizować

,BORO,BLOCK,EASEMENT,OWNER,BLDGCL,TAXCLASS,LTFRONT,LTDEPTH,EXT,STORIES,...,YEAR,Borough,Latitude,Longitude,Community Board,Council District,Census Tract,BIN,NTA,New Georeferenced Column
0,1,179,NaN,155 FRANKLIN ST LLC,RK,4,0,0,NaN,6.0,...,2018/19,MANHATTAN,40.719232,-74.008095,101.0,1.0,33.0,1.002013e+06,SoHo-TriBeCa-Civic Center-Little Italy,POINT (-74.008095 40.719232)
1,1,16,NaN,BROADLAR LLC,R4,2,0,0,NaN,23.0,...,2018/19,MANHATTAN,40.709270,-74.016487,101.0,1.0,31704.0,1.000053e+06,Battery Park City-Lower Manhattan,POINT (-74.016487 40.70927)
2,1,76,NaN,"HAM, RICHARD",R4,2,0,0,NaN,25.0,...,2018/19,MANHATTAN,40.708006,-74.006164,101.0,1.0,1502.0,1.087867e+06,Battery Park City-Lower Manhattan,POINT (-74.006164 40.708006)
3,1,16,NaN,"SHIM, HAE JIN",R4,2,0,0,NaN,27.0,...,2018/19,MANHATTAN,40.709506,-74.018002,101.0,1.0,31704.0,1.000061e+06,Battery Park City-Lower Manhattan,POINT (-74.018002 40.709506)
4,1,175,NaN,BRIT GIRL LLC,R4,2,0,0,NaN,19.0,...,2018/19,MANHATTAN,40.718024,-74.003903,101.0,1.0,33.0,1.001937e+06,SoHo-TriBeCa-Civic Center-Little Italy,POINT (-74.003903 40.718024)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11941,5,1670,NaN,"MORGAN, DONALD",A5,1,25,100,NaN,2.0,...,2017/18,STATEN IS,40.625493,-74.166429,501.0,49.0,30302.0,5.151984e+06,Mariner's Harbor-Arlington-Port Ivory-Granitev...,POINT (-74.166429 40.625493)
11942,5,1691,NaN,"THOMPSON, NICOLA",R3,1A,0,0,NaN,1.0,...,2017/18,STATEN IS,40.625612,-74.159455,501.0,49.0,30302.0,5.118625e+06,Mariner's Harbor-Arlington-Port Ivory-Granitev...,POINT (-74.159455 40.625612)
11943,5,1696,NaN,GREGORY E MCNEIL,A2,1,45,100,NaN,1.0,...,2017/18,STATEN IS,40.624666,-74.155115,501.0,49.0,30302.0,5.033624e+06,Mariner's Harbor-Arlington-Port Ivory-Granitev...,POINT (-74.155115 40.624666)
11944,5,1985,NaN,PAUL ADEST,A5,1,62,50,NaN,2.0,...,2017/18,STATEN IS,40.599127,-74.134083,502.0,50.0,27302.0,5.034388e+06,New Springville-Bloomfield-Travis,POINT (-74.134083 40.599127)
